In [1]:
import pandas as pd
from src.config import load_config
from src.universe import Universe
from src.data_providers.yfinance_provider import YFinanceProvider
from src.data_providers.fred_provider import FredProvider
from src.data_providers.nav_provider import NavProvider
from src.signals.premium_discount import calculate_premium_discount_signal
from src.signals.volume_liquidity import calculate_volume_liquidity_signal
from src.signals.divergence import calculate_divergence_signal
from src.signals.cross_asset_stress import calculate_cross_asset_signal

In [2]:
config = load_config("config/test.yaml")
universe = Universe(config)
cache_dir = getattr(config, 'cache_dir', 'data/cache')
yf = YFinanceProvider(cache_dir=cache_dir)
fred = FredProvider(cache_dir=cache_dir)
nav = NavProvider(yf)

In [7]:
start_date = "2025-01-01"
end_date = "2025-11-27"
tickers = universe.valid_tickers
fetch_start = pd.to_datetime(start_date) - pd.Timedelta(days=200)
fetch_start_str = fetch_start.strftime('%Y-%m-%d')
ohlcv = yf.fetch_ohlcv(tickers, fetch_start_str, end_date)
closes = ohlcv.xs('Close', level=1, axis=1)

In [8]:
print(tickers)
print(ohlcv.head())
print(closes.head())

['VTI', 'BND']
                   VTI                                               \
Price             Open        High         Low       Close   Volume   
Date                                                                  
2024-06-17  261.360888  264.294211  260.997905  263.627106  2452000   
2024-06-18  263.764476  264.431612  263.568285  264.294250  2024300   
2024-06-20  264.823985  265.069247  262.675488  263.627106  3168600   
2024-06-21  263.332754  263.715347  262.390927  263.224823  2676000   
2024-06-24  263.254300  264.667011  262.704916  262.783386  2226500   

                  BND                                               
Price            Open       High        Low      Close      Volume  
Date                                                                
2024-06-17  68.578272  68.616178  68.502462  68.597221   4019000.0  
2024-06-18  68.682531  68.928904  68.673053  68.834145   8491900.0  
2024-06-20  68.606718  68.729904  68.559335  68.701477   4227800.0  
20

In [9]:
# Fetch NAV/Benchmarks
bench_map = universe.get_benchmark_map()
navs = nav.get_nav_or_proxy(
  tickers, bench_map, fetch_start_str, end_date)

# Fetch Credit/VIX
credit = fred.get_credit_spreads(fetch_start_str)
vix_df = yf.fetch_single_ticker("^VIX", fetch_start_str, end_date)
vix = vix_df['Close'] if not vix_df.empty else pd.Series(dtype=float)

In [12]:
print(bench_map, navs.head())

{'VTI': 'SPY', 'BND': 'AGG'}                    VTI        BND
Date                             
2024-06-17  537.142700  92.399269
2024-06-18  538.507385  92.730736
2024-06-20  537.044556  92.569748
2024-06-21  536.324646  92.579216
2024-06-24  534.581116  92.654961


In [14]:
print(credit.head())

            IG_OAS  HY_OAS
2024-06-17    0.95    3.26
2024-06-18    0.96    3.24
2024-06-19    0.96    3.24
2024-06-20    0.96    3.23
2024-06-21    0.96    3.21


In [19]:
print(vix_df.head(10))

Price        Open   High    Low  Close  Volume
Date                                          
2024-06-17  13.07  13.29  12.50  12.75       0
2024-06-18  12.70  12.74  12.24  12.30       0
2024-06-20  12.50  13.55  12.18  13.28       0
2024-06-21  13.22  13.78  12.99  13.20       0
2024-06-24  13.85  13.88  13.15  13.33       0
2024-06-25  13.48  13.52  12.84  12.84       0
2024-06-26  12.81  13.24  12.37  12.55       0
2024-06-27  12.69  12.77  12.21  12.24       0
2024-06-28  12.24  12.76  11.87  12.44       0
2024-07-01  12.98  13.26  12.10  12.22       0


In [20]:
# Premium/Discount Z-Scores
pd_z = calculate_premium_discount_signal(
    closes, navs,
    window=config.windows.get('premdisc', 126),
    threshold_z=config.thresholds.get('premdisc_z', 2.0),
    return_series=True
)

# Volume Z-Scores
vol_z = calculate_volume_liquidity_signal(
    ohlcv,
    vol_window=config.windows.get('volume', 20),
    range_window=config.windows.get('range', 60),
    threshold_vol_z=config.thresholds.get('volume_z', 2.0),
    threshold_range_z=config.thresholds.get('range_z', 2.0),
    return_series=True
)

# Divergence Z-Scores
div_z = calculate_divergence_signal(
    closes, navs,
    window=config.windows.get('divergence', 126),
    threshold_z=config.thresholds.get('divergence_z', 2.0),
    return_series=True
)

# Cross Asset Stress
stress_series = calculate_cross_asset_signal(
    credit, vix,
    oas_jump_bps=config.thresholds.get('oas_jump_bps', 15),
    stress_z=config.thresholds.get('stress_z', 2.0),
    vix_z=config.thresholds.get('vix_z', 2.0),
    return_series=True
)

In [31]:
common_index = closes.index.intersection(navs.index)
prices = closes.loc[common_index]
navs = navs.loc[common_index]
prem_disc = (prices - navs) / navs
print(prem_disc.tail())
window=config.windows.get('premdisc', 126)
rolling_mean = prem_disc.rolling(window=window).mean()
print(rolling_mean.tail())
rolling_std = prem_disc.rolling(window=window).std()
z_scores = (prem_disc - rolling_mean) / rolling_std
print(z_scores.tail())

                 VTI       BND
Date                          
2025-11-20 -0.509448 -0.258084
2025-11-21 -0.508672 -0.258065
2025-11-24 -0.508561 -0.258247
2025-11-25 -0.507926 -0.257738
2025-11-26 -0.507945 -0.258122
                 VTI       BND
Date                          
2025-11-20 -0.507635 -0.257936
2025-11-21 -0.507642 -0.257940
2025-11-24 -0.507646 -0.257947
2025-11-25 -0.507646 -0.257947
2025-11-26 -0.507643 -0.257950
                 VTI       BND
Date                          
2025-11-20 -2.470260 -0.453432
2025-11-21 -1.393237 -0.383603
2025-11-24 -1.229889 -0.933404
2025-11-25 -0.376331  0.651138
2025-11-26 -0.407659 -0.534413
